In [1]:
import nltk
nltk.download("stopwords")
nltk.download('punkt')

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data] Unzipping corpora/stopwords.zip.
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data] Unzipping tokenizers/punkt.zip.
 Out[ 1 ]: True

In [2]:
from nltk.tokenize import sent_tokenize, word_tokenize
from nltk.corpus import stopwords
from collections import Counter
import pandas as pd
import time

def clean_data(str):
    wordsFiltered = []
    stopWords = set(stopwords.words('english'))
    words = word_tokenize(str)
    for w in words:
        if w not in stopWords and len(w)!=1:
            wordsFiltered.append(w)
    return " ".join(wordsFiltered)

In [3]:
articles = pd.read_csv("/dbfs/FileStore/tables/proyecto3/articles1.csv", usecols=[1,2,9])
articles['content'] = articles['content'].apply(clean_data)

In [4]:
articles2 = pd.read_csv("/dbfs/FileStore/tables/proyecto3/articles2.csv", usecols=[1,2,9])
articles2['content'] = articles2['content'].apply(clean_data)

In [5]:
articles3 = pd.read_csv("/dbfs/FileStore/tables/proyecto3/articles3.csv", usecols=[1,2,9])
articles3['content'] = articles3['content'].apply(clean_data)

In [6]:
articlesFinal = pd.concat([articles, articles2, articles3])

In [7]:
articlesFinal['content']= articlesFinal['content'].str.lower()
articlesFinal['title'] = articlesFinal['title'].str.lower()

In [8]:
articlesFinal.to_csv('/dbfs/FileStore/tables/proyecto3/results.csv',index=False,header=False, sep=';')

In [9]:
# Asi se lee 
articles4 = pd.read_csv("/dbfs/FileStore/tables/proyecto3/results.csv", sep=';')

In [10]:
invertedIndex = dict()
titles = dict()

#Contador de palabras por articulo que crea un diccionario
def word_count(line,id):    
    counts = dict()
    words = line.split(' ')
    for word in words: 
      if word in invertedIndex and id in invertedIndex[word]:
        invertedIndex[word][id] += 1
      elif word in invertedIndex and id not in invertedIndex[word]:
        invertedIndex[word].update({id:1})
      else:
        invertedIndex[word] = {id:1}

#Diccionario de títulos (opcional)
def titleDict(id,title):
   titles[id] = title
    
#Se pasa cada articulo para hacer el indice
for index, row in articlesFinal.iterrows():
    word_count(row["content"],row["id"])
    titleDict(row["id"],row["title"])
    

In [11]:
import operator

#Método para buscar en el indice invertido
def searchWord(word):
  if(word.lower() not in invertedIndex):
    print("Word not found")
    return
  articles = list(invertedIndex[word.lower()].items())
  articles.sort(key = lambda x: x[1], reverse = True) 
  
  for id,freq in articles[:5]:
    print(freq, id, titles[id])

In [12]:
searchWord("house")

126 78594 a sit-in on the house floor over gun control 
86 207354 full transcript: sally yates and james clapper testify on russian election interference
63 108997 who benefits from the tiny house revolution?
45 202036 nancy pelosi says a clinton landslide could put the house in play. the math backs her up.
45 28005 exclusive — audio emerges of when paul ryan abandoned donald trump: ‘i am not going to defend donald trump—not now, not in the future’

In [13]:
["hola",2,3,4,5,6]